# Termin 2 - Machine Learning Algorithmen (Aufgabe)

## Einführung in das Maschinelle Lernen  
### Überblick: Überwachtes vs. Unüberwachtes Lernen

Maschinelles Lernen ist ein Teilbereich der künstlichen Intelligenz. Dabei lernt ein Modell aus vorhandenen Daten, um Vorhersagen oder Entscheidungen zu treffen – ohne explizit programmiert zu werden.

---

### Überwachtes Lernen (Supervised Learning)

Beim überwachten Lernen wird ein Modell mit **gelabelten Trainingsdaten** trainiert. Das bedeutet: Die Daten enthalten sowohl Eingabewerte als auch Zielwerte (Labels), anhand derer das Modell lernt.

#### Typische Aufgaben:

- **Klassifikation**  
  Vorhersage einer Kategorie, z. B.:
  - Binär: Ja/Nein, Krank/Gesund
  - Mehrklassen: Hund, Katze, Vogel

- **Regression**  
  Vorhersage eines kontinuierlichen Werts, z. B.:  
  Grundwasserstand in Abhängigkeit von Niederschlag und Landnutzung

#### Häufig verwendete Algorithmen:

- Logistische Regression  
- Entscheidungsbäume (Decision Trees)  
- Random Forest  
- Support Vector Machines (SVM)

---

### Unüberwachtes Lernen (Unsupervised Learning)

Hier gibt es **keine Zielwerte**. Das Modell versucht eigenständig, Muster oder Strukturen in den Daten zu erkennen.

#### Typische Methoden:

- K-Means-Clustering: Gruppiert ähnliche Datenpunkte  
- Principal Component Analysis (PCA): Reduziert die Anzahl der Merkmale (Dimensionsreduktion)

---

Weitere Informationen zu den in `scikit-learn` verfügbaren Algorithmen für überwachtes Lernen findet ihr unter:  
https://scikit-learn.org/stable/supervised_learning.html


# Teil 1 – Überwachtes Lernen

### Binäre Klassifikation

Bei der binären Klassifikation wird ein Modell darauf trainiert, zwischen genau **zwei Klassen** zu unterscheiden.  
Beispiel: Vorhersage, ob ein Patient krank oder gesund ist (Ja/Nein).

---

### Datensatz: Einlesen und Vorbereitung

- **Datenbasis**: Grundwasser-Stichtagsmessungen aus dem Jahr 2005  
- **Zielvariable**: Landnutzung – Vorhersage, ob landwirtschaftlich genutzt (Ja/Nein)  
- **Problemtyp**: Binäre Klassifikation basierend auf Umweltmessdaten


### Vorbereitung: Wichtige Bibliotheken importieren

In [ ]:
# --- Grundlagen ---
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# --- Datenvorverarbeitung ---
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import StandardScaler

# --- Klassifikationsmodelle ---
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

# --- Bewertung des Modells ---
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# --- Pipeline zur Modellierung ---
from sklearn.pipeline import Pipeline


In [ ]:
# Einladen der Daten
filename = 'gwdata_2005.csv'
dataset = pd.read_csv(filename, delimiter=';', encoding="ISO-8859-1")

# Preprocessing (Bereinigen von NaN-Werten und Entfernen von nicht benötigten Spalten)
dataset_cleaned = dataset.dropna(axis=0, how='any')
data = dataset_cleaned.drop(['GWNum','Messstelle','Rechtswert', 'Hochwert', 'Aquifer','Aquifer2','landuse'], axis=1)
data.head()

### Explorative Übersicht über den Datensatz

In [ ]:
# Überblick über Struktur, Inhalte und Korrelationen
print("Form des Datensatzes:", data.shape)
print("\nDatentypen:")
print(data.dtypes)

print("\nStatistische Kennzahlen:")
print(data.describe())

print("\nEindeutige Werte pro Spalte:")
print(data.nunique())

print("\nFehlende Werte pro Spalte:")
print(data.isnull().sum())

print("\nKorrelationen (numerisch):")
print(data.corr(numeric_only=True))

### Zielvariable: `landuse_num`

Die Spalte `landuse_num` ist die Zielvariable für die Klassifikation. Sie kodiert landwirtschaftliche Nutzung binär:

- `0`: keine Landwirtschaft (`no_agr`)
- `1`: landwirtschaftlich genutzt (`agr`)


In [ ]:
print("Anzahl der Klassen in 'landuse_num':")
print(data['landuse_num'].value_counts().rename(index={0: 'Keine Landwirtschaft', 1: 'Landwirtschaft'}))


### Aufteilen in Trainings- und Testdaten

Um die Modellleistung objektiv zu bewerten, wird der Datensatz in zwei Teile getrennt:

- **Trainingsdaten**: zum Trainieren des Modells
- **Testdaten**: zur Überprüfung der Generalisierungsfähigkeit

Typisches Verhältnis: 80 % Training, 20 % Test (Standardwert).



In [ ]:
# Zufalls-Seed für Reproduzierbarkeit
random_state = 42

# Eingabedaten (X) und Zielvariable (y)
X = data.drop('landuse_num', axis=1)
y = data['landuse_num']

# Aufteilen in Trainings- und Testdaten (80/20)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=random_state, shuffle=True
)

# Formen der Datensätze anzeigen
print("X_train:", X_train.shape)
print("X_test: ", X_test.shape)
print("y_train:", y_train.shape)
print("y_test: ", y_test.shape)


### Support Vector Machines (SVM)

Support Vector Machines sind leistungsfähige Verfahren für Klassifikation und Regression.  
Sie arbeiten, indem sie eine **Entscheidungsgrenze** (Hyperebene) im Merkmalsraum bestimmen, die die Klassen bestmöglich voneinander trennt.

#### Support Vector Classification (SVC)

`SVC` ist die SVM-Variante für Klassifikationsaufgaben. Sie eignet sich besonders für komplexe, auch nicht-linear trennbare Probleme – etwa durch den Einsatz von Kernfunktionen (Kernels).

---

Weitere Informationen in der scikit-learn-Dokumentation:

- [SVM in scikit-learn](https://scikit-learn.org/stable/modules/svm.html)  
- [Modellbewertung & Scoring](https://scikit-learn.org/stable/modules/model_evaluation.html)


In [ ]:
# Reproduzierbarkeit sicherstellen
random_state = 88

# Bewertungsmetrik für Cross-Validation
scoring = 'accuracy'

# Anzahl der Folds für K-Fold-Validierung
n_splits = 10


### 1/3: SVC mit unskalierten Eingabedaten

In diesem Schritt wird ein `SVC`-Modell (Support Vector Classification) mit den unskalierten Eingabedaten trainiert.

Support Vector Machines sind empfindlich gegenüber ungleich skalierten oder unterschiedlich verteilten Merkmalen.  
Fehlende Standardisierung kann die Modellgüte deutlich beeinträchtigen. Dieses Modell dient daher als Referenz für spätere Vergleiche mit skalierten Varianten.

[Mehr zur SVC in der scikit-learn-Dokumentation](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC)


In [ ]:
# SVM-Modell mit RBF-Kernel (ohne Skalierung)
svm_raw = SVC(kernel='rbf', gamma='auto')

# K-Fold-Setup
kfold = KFold(n_splits=n_splits, random_state=random_state, shuffle=True)

# Kreuzvalidierung
cv_scores_svm_raw = cross_val_score(svm_raw, X_train, y_train, cv=kfold, scoring=scoring)

# Mittelwert und Standardabweichung der Genauigkeit
mean_accuracy_raw = cv_scores_svm_raw.mean()
std_accuracy_raw = cv_scores_svm_raw.std()

# Ergebnis ausgeben
print(f"SVM Accuracy (ohne Skalierung): {mean_accuracy_raw:.3f} ± {std_accuracy_raw:.3f}")

### 2/3: SVC mit skalierten Eingabedaten

Durch Standardisierung (z. B. mit `StandardScaler`) werden alle Merkmale auf denselben Wertebereich gebracht.  
Dies kann die Leistung von SVMs erheblich verbessern.

Im Folgenden wird dasselbe `SVC`-Modell wie zuvor verwendet, diesmal jedoch mit vorgeschalteter Skalierung der Eingabedaten.


In [ ]:
# Erstellen und Anwenden des StandardScalers auf die Trainingsdaten
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# SVM-Modell mit RBF-Kernel (auf skalierten Daten)
svm_scaled = SVC(kernel='rbf', gamma='auto')

# K-Fold-Cross-Validation definieren
kfold = KFold(n_splits=n_splits, random_state=random_state, shuffle=True)

# Kreuzvalidierung mit skalierten Daten
cv_scores_svm_scaled = cross_val_score(svm_scaled, X_train_scaled, y_train, cv=kfold, scoring=scoring)

# Genauigkeit und Streuung berechnen
mean_accuracy_scaled = cv_scores_svm_scaled.mean()
std_accuracy_scaled = cv_scores_svm_scaled.std()

# Ausgabe
print(f"SVM Accuracy (mit Skalierung): {mean_accuracy_scaled:.3f} ± {std_accuracy_scaled:.3f}")


<div style="background-color:#e6f0ff; padding:10px; border-radius:5px; font-weight:bold">
   For Your Interest: Pipeline mit mehreren Verarbeitungsschritten
</div>



Der folgende Abschnitt zeigt, wie man mithilfe von `Pipeline` mehrere Verarbeitungsschritte nahtlos kombiniert – von der Vorverarbeitung bis zur Modellierung mit einem SVM.

**Typische Komponenten einer solchen Pipeline:**

- **Imputation**:  
  `SimpleImputer(strategy='mean')`  
  → Ersetzt fehlende Werte durch den Mittelwert der jeweiligen Spalte.

- **Skalierung**:  
  `StandardScaler()`  
  → Skaliert alle Merkmale auf dieselbe Größenordnung.

- **Feature Engineering (optional)**:  
  `PolynomialFeatures(degree=2)`  
  → Fügt polynomiale Merkmale hinzu, um nichtlineare Zusammenhänge abzubilden.

- **Feature Selection**:  
  `SelectKBest(f_classif, k=10)`  
  → Wählt die 10 Merkmale mit der höchsten Relevanz anhand des ANOVA-F-Tests aus.

- **Dimensionsreduktion**:  
  `PCA(n_components=0.95)`  
  → Reduziert die Dimension, wobei 95 % der Varianz erhalten bleiben.

- **Klassifikator**:  
  `SVC(kernel='rbf', gamma='auto')`  
  → SVM mit RBF-Kernel für die Klassifikation.

---

> **Hinweis:**  
> Der folgende Code dient der Veranschaulichung.  
> Einige der verwendeten Komponenten sind möglicherweise **nicht in der aktuellen Umgebung installiert** und daher **nicht direkt ausführbar**.
```
pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler()),
    ('poly', PolynomialFeatures(degree=2)),
    ('feature_select', SelectKBest(f_classif, k=10)),
    ('pca', PCA(n_components=0.95)),
    ('svm', SVC(kernel='rbf', gamma='auto'))
])
```


In [ ]:
# Pipeline: Skalierung + SVM-Modell
svm_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('svm', SVC(kernel='rbf', gamma='auto'))
])

# K-Fold-Cross-Validation definieren
kfold = KFold(n_splits=n_splits, random_state=random_state, shuffle=True)

# Kreuzvalidierung mit der Pipeline
cv_scores_svm_pipeline = cross_val_score(svm_pipeline, X_train, y_train, cv=kfold, scoring=scoring)

# Genauigkeit berechnen
mean_accuracy_pipeline = cv_scores_svm_pipeline.mean()
std_accuracy_pipeline = cv_scores_svm_pipeline.std()

# Ausgabe der mittleren Genauigkeit ± Standardabweichung
print(f"SVM Accuracy (Pipeline): {mean_accuracy_pipeline:.3f} ± {std_accuracy_pipeline:.3f}")


### 3/3: Random Forest Classifier

Der `RandomForestClassifier` ist ein Ensemble-Lernverfahren, das auf einer Vielzahl von Entscheidungsbäumen basiert.  
Durch das sogenannte Bagging (Bootstrap Aggregation) und die zufällige Auswahl von Features bei jedem Split werden Überanpassung reduziert und die Generalisierungsfähigkeit erhöht.

Vorteile:
- Robust gegenüber Ausreißern und Rauschen
- Kaum anfällig für Overfitting bei ausreichender Baumanzahl
- Gut skalierbar und vielseitig einsetzbar (Klassifikation und Regression)

[Weitere Informationen zum `RandomForestClassifier` in scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier)


In [ ]:
# Random Forest-Modell mit Standardparametern
rf = RandomForestClassifier(random_state=random_state)

# K-Fold-Cross-Validation definieren
kfold = KFold(n_splits=n_splits, random_state=random_state, shuffle=True)

# Kreuzvalidierung mit skalierten Eingabedaten
cv_scores_rf = cross_val_score(rf, X_train_scaled, y_train, cv=kfold, scoring=scoring)

# Genauigkeit und Standardabweichung berechnen
mean_accuracy_rf = cv_scores_rf.mean()
std_accuracy_rf = cv_scores_rf.std()

# Ausgabe
print(f"Random Forest Accuracy: {mean_accuracy_rf:.3f} ± {std_accuracy_rf:.3f}")


### Vergleich der Ergebnisse

Im Folgenden werden die mittleren Genauigkeiten der drei Modelle gegenübergestellt:

- **SVC ohne Skalierung**
- **SVC mit Skalierung**
- **Random Forest Classifier**

Der Vergleich dient zur Veranschaulichung der Auswirkungen von Vorverarbeitung und Modellwahl auf die Klassifikationsgenauigkeit.


In [ ]:
# Ergebnisse aus Cross-Validation (Accuracy-Werte pro Fold)
results = [
    cv_scores_svm_raw,        # SVM Unskaliert
    cv_scores_svm_scaled,      # SVM mit manueller Skalierung   
    cv_scores_rf,             # Random Forest
]

# Boxplot zum Vergleich der Modelle
plt.figure(figsize=(8, 6))
plt.boxplot(results, patch_artist=True)
plt.title('Vergleich der Modelle (Kreuzvalidierung)')
plt.xticks([1, 2, 3], ['SVM', 'SVM (scaled)', 'Random Forest'])
plt.ylabel('Accuracy')
plt.grid(True)
plt.show()


### Auswertung des besten Modells

Nach dem Vergleich der Modelle anhand ihrer Kreuzvalidierungsergebnisse wird in diesem Abschnitt das Modell mit der höchsten mittleren Genauigkeit näher ausgewertet.

Die folgenden Analyseschritte können dabei durchgeführt werden:

- Training des Modells auf dem vollständigen Trainingsdatensatz
- Vorhersage auf dem Testdatensatz (`X_test`)
- Berechnung relevanter Metriken:
  - Konfusionsmatrix
  - Genauigkeit, Präzision, Recall, F1-Score
- Optional: Visualisierung der Konfusionsmatrix


In [ ]:
# Modell für finale Auswertung auf dem Testdatensatz
rf_final = RandomForestClassifier(random_state=random_state)

# Skalierung der Testdaten mit dem bereits auf Trainingsdaten gefitteten Scaler
X_test_scaled = scaler.transform(X_test)

# Training des Random Forest auf dem gesamten (skalierten) Trainingsdatensatz
rf_final.fit(X_train_scaled, y_train)

# Vorhersage auf dem (skalierten) Testdatensatz
y_pred = rf_final.predict(X_test_scaled)


In [ ]:
# Genauigkeit auf dem Testdatensatz berechnen
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Konfusionsmatrix berechnen
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

# Klassifikationsbericht mit Precision, Recall, F1-Score
class_report = classification_report(y_test, y_pred)
print("Classification Report:")
print(class_report)


**Ergebnis:**  
Das Modell erkennt Klasse 0 (*keine Landwirtschaft*) sehr gut (**Recall: 89 %**).  
Bei Klasse 1 (*Landwirtschaft*) gibt es größere Probleme:  
Die **Precision** (Wie zuverlässig ist das Modell, wenn es „1“ sagt) ist gut,  
der **Recall** (Wie viele echte „1“ wurden erkannt) jedoch nur mittelmäßig.  
Der **F1-Score** (Kompromiss zwischen beiden) liegt entsprechend im mittleren Bereich.


---

### Multiklassen-Klassifikation
Viele reale Klassifikationsprobleme bestehen nicht nur aus zwei, sondern aus mehreren Klassen.  
Multiklassen-Klassifikation bezeichnet die Vorhersage einer von mehr als zwei möglichen Zielklassen.
#### IRIS-Datensatz laden

Der Iris-Datensatz enthält 150 Beispiele von Blumen, die jeweils einer von drei Arten angehören (*Setosa*, *Versicolor* oder *Virginica*).  
Er besteht aus vier numerischen Merkmalen: Länge und Breite von Kelchblatt (*Sepal*) und Kronblatt (*Petal*).

Dieser Datensatz eignet sich gut, um verschiedene Machine-Learning-Modelle hinsichtlich ihrer Fähigkeit zur Klassifikation mehrerer Klassen zu testen.

<div style="text-align:center">
  <img src="https://s3.amazonaws.com/assets.datacamp.com/blog_assets/Machine+Learning+R/iris-machinelearning.png" width="75%">
</div>

Der Datensatz wird als sogenanntes `Bunch`-Objekt geladen. Dieses enthält neben den Eingabedaten auch Zusatzinformationen wie z. B.:

- **data**: Ein NumPy-Array mit den Merkmalswerten.
- **target**: Ein NumPy-Array mit den Zielwerten (0, 1, 2).
- **feature_names**: Namen der Merkmale (z. B. `sepal length`).
- **target_names**: Namen der Zielklassen (`setosa`, `versicolor`, `virginica`).
- **DESCR**: Beschreibung des Datensatzes.


In [ ]:
from sklearn.datasets import load_iris

# Laden des Iris-Datensatzes
iris = load_iris()

# Extrahieren der Merkmale und des Targets aus dem Iris-Datensatz
X = iris.data  # Merkmale
y = iris.target  # Zielwerte
#print(iris.DESCR)

## Aufgabe: Multiklassen-Klassifikation mit dem Iris-Datensatz

Führe eine Multiklassen-Klassifikation mit dem Iris-Datensatz durch.  
Implementiere mindestens zwei Modelle:

- Ein Modell auf Basis von **Support Vector Machines (SVM)**  
  (einmal **ohne Skalierung** und einmal **mit Skalierung**)
- Ein Modell auf Basis von **Random Forest (RF)**

### Vorgehensweise:
- Teile die Daten in Trainings- und Testdatensätze auf
- Identifiziere relevante Merkmale für die Klassifikation
- Implementiere die beiden Modelle
- Führe eine Evaluierung mit den Testdaten durch
- Berechne die Genauigkeit der Vorhersagen
- Vergleiche die Ergebnisse beider Modelle
- Diskutiere, welches Modell besser geeignet ist

---

### Bearbeitungsübersicht

- [x] Laden des Iris-Datensatzes  
- [ ] Aufteilen der Daten in Trainings- und Testdatensätze  
- [ ] Identifikation relevanter Merkmale  
- [ ] Implementierung eines SVM-Modells (unskaliert & skaliert)  
- [ ] Implementierung eines Random Forest-Modells  
- [ ] Evaluierung und Berechnung der Genauigkeit  
- [ ] Vergleich und Diskussion der Ergebnisse  

---

### Hinweis:
Achte auf eine angemessene Datenvorverarbeitung, insbesondere die **Skalierung der Eingabedaten** für SVM.  
Teste außerdem unterschiedliche Hyperparameter, um die Modellleistung zu optimieren.


### 1. Daten aufteilen

- Erstelle X_train, X_test, Y_train, Y_test.
- Definiere tesize und den random seed

In [ ]:

# Aufteilen in Trainings- und Testdaten (80/30)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

print("Trainingsdaten:", X_train.shape)
print("Testdaten:", X_test.shape)


### 2. SVM-Modell (ohne Skalierung)
- Erstelle Modell
- Definiere K-Fold CV
- Berechne und Speichere mittlere Genauigkeit und Standardabweichung

In [ ]:
# SVM ohne Skalierung
svm_raw = SVC(kernel='rbf', gamma='scale')
svm_raw.fit(X_train, y_train)
y_pred_svm_raw = svm_raw.predict(X_test)

# Auswertung
acc_svm_raw = accuracy_score(y_test, y_pred_svm_raw)
print(f"SVM Accuracy (ohne Skalierung): {acc_svm_raw:.3f}")


### 3. SVM-Modell (mit Skalierung)
- Erstelle Modell
- Definiere K-Fold CV
- Berechne und Speichere mittlere Genauigkeit und Standardabweichung


In [ ]:
# Skalierung
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# SVM mit Skalierung
svm_scaled = SVC(kernel='rbf', gamma='scale')
svm_scaled.fit(X_train_scaled, y_train)
y_pred_svm_scaled = svm_scaled.predict(X_test_scaled)

# Auswertung
acc_svm_scaled = accuracy_score(y_test, y_pred_svm_scaled)
print(f"SVM Accuracy (mit Skalierung): {acc_svm_scaled:.3f}")


### 4. Random Forest-Modell
- Erstelle Modell
- Definiere K-Fold CV
- Berechne und Speichere mittlere Genauigkeit und Standardabweichung

In [ ]:
# Random Forest (braucht keine Skalierung)
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

# Auswertung
acc_rf = accuracy_score(y_test, y_pred_rf)
print(f"Random Forest Accuracy: {acc_rf:.3f}")


### 5. Vergleich der Modellgenauigkeiten

- Führe Ergebnisse zusammen und Plotte sie

In [ ]:
print("Modellvergleich:")
print(f"- SVM (ohne Skalierung):     {acc_svm_raw:.3f}")
print(f"- SVM (mit Skalierung):      {acc_svm_scaled:.3f}")
print(f"- Random Forest:             {acc_rf:.3f}")

---

## Regression

### Concrete-Datensatz

Der Concrete-Datensatz enthält verschiedene Eigenschaften von Betonmischungen und deren resultierende Druckfestigkeit.  
Er umfasst **1.030 Datensätze** mit **8 Eingabevariablen** und einer Zielgröße (Regression):

**Eingabevariablen:**
- Cement (*Zement*)
- Blast Furnace Slag (*Hüttenzement*)
- Fly Ash (*Flugasche*)
- Water (*Wasser*)
- Superplasticizer (*Fließmittel*)
- Coarse Aggregate (*Grobgestein*)
- Fine Aggregate (*Feingestein*)
- Age (*Alter in Tagen*)

**Zielvariable:**
- Concrete Compressive Strength (*Druckfestigkeit des Betons*, in MPa)



In [ ]:
# Concrete-Datensatz laden
data = pd.read_csv('Concrete_Data.csv', sep=';', encoding='ISO-8859-1')

# Vorschau auf die ersten 5 Zeilen
data.head()

### Hinweis zur Modellbewertung (Regression)

Da wir nun eine **Regression** durchführen (nicht Klassifikation), verwenden wir eine andere Metrik zur Bewertung der Modellgüte: den **R²-Score**.

Der R²-Score (Bestimmtheitsmaß) gibt an, wie gut die vorhergesagten Werte zu den tatsächlichen Werten passen.  
Ein Wert von 1 bedeutet perfekte Vorhersage, 0 entspricht dem Mittelwertsmodell, negative Werte deuten auf schlechte Modellanpassung hin.

Wir verwenden den `r2_score` aus `sklearn.metrics`.

Weitere Informationen und alternative Regressionsmetriken wie MAE oder RMSE findest du unter:  
[scikit-learn.org – Model Evaluation (Regression)](https://scikit-learn.org/stable/modules/model_evaluation.html#regression-metrics)


In [ ]:
from sklearn.metrics import r2_score

# Einstellungen für Modellvergleich und Reproduzierbarkeit

# Zufalls-Seed für Reproduzierbarkeit der Ergebnisse
random_seed = 88

# Bewertungsmetrik für Regressionsmodelle
scoring_metric = 'r2'  # R² (Bestimmtheitsmaß)

# Anzahl der Folds für K-Fold Cross-Validation
num_folds = 10


## Preprocessing


In [ ]:
# Setzen des Seeds für die Reproduzierbarkeit
random_state = 42

# Aufteilen der Daten in Trainings- und Testdaten
X = data.drop('Druckfestigkeit', axis=1)  # Spalte Zielvariable ab
Y = data['Druckfestigkeit']  # Definiere 'landuse_num' als Zielvariable
test_size = 0.2  # Anteil der Testdaten: 20%

# Mischen und Aufteilen der Daten in Trainings- und Testdaten
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size, random_state=random_state, shuffle=True)

print("Form der Trainingsdaten (X_train):", X_train.shape)
print("Form der Testdaten (X_test):", X_test.shape)
print("Form der Trainingszielvariablen (Y_train):", Y_train.shape)
print("Form der Testzielvariablen (Y_test):", Y_test.shape)

<span style="color: red; font-size: 24px"><strong>Aufgabe:</strong></span>  
Entwickle zwei verschiedene Modelle zur Vorhersage der **Druckfestigkeit** des Betons:  
– eines basierend auf **Support Vector Machines (SVM)**  
– eines basierend auf **Random Forest (RF)**

### Schritte:
- [x] **Daten aufteilen:** Teile die Daten in Trainings- und Testdatensätze auf  
- [ ] **Modelle implementieren:** Implementiere ein SVM-Modell und ein Random Forest-Modell  
- [ ] **Trainieren & evaluieren:** Trainiere die Modelle und evaluiere sie auf den Testdaten  
- [ ] **Genauigkeit berechnen:** Ermittle die Vorhersagegüte (z. B. R² oder MAE)  
- [ ] **Ergebnisse vergleichen:** Vergleiche beide Modelle und beurteile ihre Eignung

---

### Nützliche Links:

- [Scikit-learn Übersicht](https://scikit-learn.org)  
- [SVR – Support Vector Regression](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html)  
- [RandomForestRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html)  
- [Model Evaluation in sklearn](https://scikit-learn.org/stable/modules/model_evaluation.html)


**<span style="font-size: 24px">Support Vector Regressor</span>**

**<span style="font-size: 18px">Support Vector Regressor (ohne skalierung)</span>**

- Erstelle Modell
- Definiere K-Fold CV
- Berechne und Speichere mittlere Genauigkeit und Standardabweichung

**<span style="font-size: 18px">Support Vector Regressor skaliert</span>**

- Erstelle Modell
- Definiere K-Fold CV
- Berechne und Speichere mittlere Genauigkeit und Standardabweichung

**<span style="font-size: 18px">Random Forest</span>**

- Erstelle Modell
- Definiere K-Fold CV
- Berechne und Speichere mittlere Genauigkeit und Standardabweichung

<span style="font-size: 18px">**Vergleich der Ergebnisse:**</span>

---

## Unüberwachtes Lernen

Die **PCA** (Principal Component Analysis) ist ein Verfahren zur **Dimensionsreduktion**. Sie reduziert die Anzahl der Variablen, während die **wichtigste Varianz im Datensatz erhalten bleibt**.  
Beim Iris-Datensatz kann PCA verwendet werden, um die vier Merkmale (Sepal-Länge/-Breite, Petal-Länge/-Breite) in zwei Hauptkomponenten zu überführen. Das erleichtert die Visualisierung und zeigt Muster im Datenraum auf.

**KMeans** ist ein klassischer **Clustering-Algorithmus**. Er teilt Datenpunkte in **k Cluster**, wobei jedes Cluster durch seinen **Zentroid (Mittelpunkt)** beschrieben wird.  
Im Iris-Datensatz können mit KMeans Cluster gebildet werden, die evtl. den drei Iris-Arten ähneln – ohne dass die Labels bekannt sein müssen.

---

### KMeans Clustering

#### Datensatz laden

Wir verwenden erneut den **Iris-Datensatz**, verzichten nun aber auf die Zielvariablen (`target`), da es sich um **unüberwachtes Lernen** handelt.


In [ ]:
from sklearn.datasets import load_iris
import pandas as pd

# Iris-Daten ohne Zielvariable laden
iris = load_iris()
X = iris.data
feature_names = iris.feature_names

# In DataFrame umwandeln und anzeigen
df_iris = pd.DataFrame(X, columns=feature_names)
print(df_iris.head())

### Merkmalskombinationen visuell vergleichen

Der Iris-Datensatz besteht aus vier numerischen Merkmalen:  
Bei vier Merkmalen ergeben sich insgesamt **6 mögliche 2D-Kombinationen**, in denen jeweils zwei Merkmale gegenübergestellt werden können.  

Diese Visualisierung zeigt alle möglichen **Paarungen der Merkmale als Streudiagramme**.  
Einzelne Gruppen oder Muster lassen sich teilweise erkennen, jedoch ist eine **klare visuelle Trennung oder Gruppierung rein durch das Auge schwierig**, insbesondere ohne Farbcodierung nach Klassen oder Cluster.

Diese Streudiagramme bieten dennoch eine erste Einschätzung möglicher Zusammenhänge oder Trennbarkeit in bestimmten Merkmalskombinationen.


In [ ]:

# Alle Kombinationen von 2 Features
pairs = [(0, 1), (0, 2), (0, 3),
         (1, 2), (1, 3), (2, 3)]

# 2x3 Subplots
fig, axes = plt.subplots(2, 3, figsize=(18, 10))

for idx, (i, j) in enumerate(pairs):
    ax = axes[idx // 3, idx % 3]
    ax.scatter(X[:, i], X[:, j], s=50, alpha=0.7)
    ax.set_xlabel(feature_names[i])
    ax.set_ylabel(feature_names[j])
    ax.set_title(f'{feature_names[i]} vs. {feature_names[j]}')
    ax.grid(True)

plt.tight_layout()
plt.show()


Da der Iris-Datensatz drei Klassen umfasst (**Setosa**, **Versicolor**, **Virginica**), ist es sinnvoll, die Daten mithilfe der **PCA** auf **drei Dimensionen** zu reduzieren.

So lässt sich ein Großteil der Varianz im Datensatz bewahren und gleichzeitig eine kompakte Repräsentation erzeugen, die sich gut für **Visualisierung** und **Clusteranalyse** eignet.


In [ ]:
from sklearn.cluster import KMeans

# KMeans-Clustering mit 3 Clustern
kmeans = KMeans(n_clusters=3, n_init=10, random_state=42)
kmeans.fit(X)

# Cluster-Zuweisung für jeden Datenpunkt
cluster_labels = kmeans.labels_


**<span style="font-size: 20px">Validierung</span>**

Für das KMeans-Clustering gibt es verschiedene Metriken zur Bewertung der Clusterqualität, darunter:

- **Inertia**
- **Silhouette Score**
- **Calinski-Harabasz Index**
- **Davies-Bouldin Index**

Diese Metriken geben Einblicke in die Trennschärfe, Kompaktheit und Struktur der erkannten Cluster.  
Details zu deren Anwendung in Scikit-Learn findest du in der  
👉 [Scikit-Learn-Dokumentation zur Clusterbewertung](https://scikit-learn.org/stable/modules/clustering.html#clustering-performance-evaluation)

---

### Inertia

Die **Inertia** misst die Summe der quadrierten Abstände aller Datenpunkte zu den Zentroiden ihrer zugewiesenen Cluster.  
Eine **niedrige Inertia** deutet auf kompakte, gut definierte Cluster hin.  
Eine **hohe Inertia** spricht eher für verstreute oder überlappende Gruppen.

Die Inertia-Formel:

$$
\text{Inertia} = \sum_{j=1}^{k} \sum_{i \in C_j} \|x_i - \mu_j\|^2
$$

Dabei ist:

- \( k \): Anzahl der Cluster  
- \( C_j \): Datenpunkte im j-ten Cluster  
- \( x_i \): einzelner Datenpunkt  
- \( \mu_j \): Schwerpunkt (Zentroid) des j-ten Clusters


In [ ]:
# Berechnung der Inertia für verschiedene Cluster-Anzahlen
inertia_values = [
    KMeans(n_clusters=k, n_init=10, random_state=42).fit(X).inertia_
    for k in range(1, 10)
]

# In DataFrame umwandeln
df_inertia = pd.DataFrame({
    'Anzahl_Cluster': range(1, 10),
    'Inertia': inertia_values
})

# Plot
plt.figure(figsize=(7, 4))
plt.plot(df_inertia['Anzahl_Cluster'], df_inertia['Inertia'], marker='o')
plt.title("Elbow-Methode zur Bestimmung der Clusteranzahl")
plt.xlabel("Anzahl der Cluster (k)")
plt.ylabel("Inertia")
plt.grid(True)
plt.xticks(range(1, 10))
plt.tight_layout()
plt.show()


In [ ]:
# Initialisiere das KMeans-Modell mit 3 Clustern
kmeans = KMeans(n_clusters=3, n_init=10, random_state=42)

# Trainiere das Modell auf dem vollständigen Iris-Datensatz (ohne Zielvariable)
kmeans.fit(X)

# Weist jedem Datenpunkt ein Clusterlabel (0, 1 oder 2) zu
y_kmeans = kmeans.predict(X)

# Extrahiere die berechneten Zentren der Cluster (Mittelwerte der Clusterpunkte)
centers = kmeans.cluster_centers_

# Funktion zur 2D-Visualisierung des Cluster-Ergebnisses anhand zweier Merkmale
def plot_kmeans_clusters(X, y_labels, centroids, x_index=0, y_index=1, feature_names=None):
    plt.figure(figsize=(6, 4))
    
    # Für jedes Cluster: Streudiagramm der zugehörigen Punkte
    for label in range(centroids.shape[0]):
        plt.scatter(
            X[y_labels == label, x_index],     # Punkte dieses Clusters auf X-Achse
            X[y_labels == label, y_index],     # Punkte dieses Clusters auf Y-Achse
            s=50,
            label=f'Cluster {label + 1}'
        )
    
    # Plot der Cluster-Zentren (Zentroide)
    plt.scatter(
        centroids[:, x_index],
        centroids[:, y_index],
        s=300,
        marker='*',
        c='black',
        label='Cluster-Zentren'
    )
    
    # Achsentitel ausgeben, ggf. mit echten Merkmalsnamen
    plt.title('KMeans-Clustering (2D-Projektion)')
    plt.xlabel(feature_names[x_index] if feature_names else f"Feature {x_index}")
    plt.ylabel(feature_names[y_index] if feature_names else f"Feature {y_index}")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# Aufruf der Plotfunktion: zeige Cluster nach Sepal Length & Sepal Width
plot_kmeans_clusters(X, y_kmeans, centers, x_index=0, y_index=1, feature_names=feature_names)


**<span style="font-size: 24px">Principle Component Analysis (PCA)</span>**

**<span style="font-size: 20px">Datensatz laden</span>**

Wir laden hier zusätzlich die Zielvariable herunter, da wir diese zum Plotten benötigen.

In [ ]:
from sklearn.datasets import load_iris

# Iris-Daten laden (Features und Zielvariable)
iris = load_iris()
X = iris.data           # Merkmalsmatrix

color = iris.target     # Klassenlabels (für Farbzuweisung beim Plotten)


**<span style="font-size: 20px">Erklärte Varianz</span>**

Die kumulierte Erklärte Varianz (Cumulative Variance Ratio) in der PCA ist ein Maß dafür, wie viel Information von den ursprünglichen Daten in den Hauptkomponenten erhalten bleibt. Es zeigt den Anteil der gesamten Varianz im Datensatz an, der von den ersten k Hauptkomponenten erklärt wird. Je höher der Wert, desto besser erfassen die Hauptkomponenten die Variationen im Datensatz.

Wenn die kumulierte erklärte Varianz 0,98 beträgt, bedeutet dies, dass die ersten k Hauptkomponenten zusammen 98 % der gesamten Varianz im Datensatz erklären. Mit anderen Worten, diese Hauptkomponenten erfassen einen Großteil der Variationen in den Daten und bieten eine gute Zusammenfassung des Datensatzes. Dies ist oft ein Hinweis darauf, dass eine Reduktion der Dimensionen auf k Hauptkomponenten eine sinnvolle Wahl sein könnte, da sie die Daten mit relativ hoher Genauigkeit darstellen können.


In [ ]:
from sklearn.decomposition import PCA
# PCA durchführen und kumulierte erklärte Varianz berechnen
pca = PCA().fit(X)
cumulative_variance_ratio = np.cumsum(pca.explained_variance_ratio_)

# Plot erstellen
plt.figure(figsize=(6, 3))
plt.plot(range(1, len(cumulative_variance_ratio) + 1), cumulative_variance_ratio, marker='o')
plt.xlabel('Anzahl der Komponenten')
plt.ylabel('Kumulierte erklärte Varianz')
plt.title('Kumulierte erklärte Varianz vs. Anzahl der PCA-Komponenten')
plt.show()


**<span style="font-size: 20px">Dimensionsreduktion</span>**

Im nächsten Schritt wird die Anzahl der Merkmale mithilfe der PCA auf zwei bzw. drei Hauptkomponenten reduziert. Ziel ist es, die Daten kompakter darzustellen und gleichzeitig möglichst viel Varianz beizubehalten – ideal für Visualisierung und Clusteranalyse.


In [ ]:
from sklearn.datasets import load_iris
from sklearn.decomposition import PCA
import pandas as pd

# Iris-Daten laden (nur Features und Zielvariable für Visualisierung)
iris = load_iris()
X = iris.data
color = iris.target

# PCA mit 3 Hauptkomponenten durchführen
pca = PCA(n_components=3)
X_pca = pca.fit_transform(X)

# In DataFrame umwandeln für bessere Übersicht
X_pca = pd.DataFrame(X_pca, columns=['PC1', 'PC2', 'PC3'])

# Formen vergleichen
print("Ursprüngliche Form der Daten:", X.shape)
print("Transformierte Form (PCA):    ", X_pca.shape)


**<span style="font-size: 20px">Visualisierung</span>**

Die Funktion `plot_combined_3d` erzeugt eine kombinierte Darstellung aus einem **3D-Scatterplot** (oben) und drei **2D-Scatterplots** (unten).

- Der 3D-Plot zeigt die Datenpunkte im Raum der ersten drei Hauptkomponenten (PC1–PC3).
- Die 2D-Plots visualisieren jeweils die Beziehung zwischen zwei dieser Komponenten: PC1 vs. PC2, PC1 vs. PC3 und PC2 vs. PC3.

Diese Darstellung hilft, Strukturen, Trennbarkeit oder potenzielle Cluster im reduzierten Merkmalsraum zu erkennen.


In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def plot_combined_3d(X_pca, color):
    # Abbildung erzeugen
    fig = plt.figure(figsize=(12, 8))
    
    # Oberer Plot: 3D-Scatterplot der ersten drei Hauptkomponenten
    ax_3d = fig.add_subplot(2, 1, 1, projection='3d')
    scatter_3d = ax_3d.scatter(
        X_pca[:, 0], X_pca[:, 1], X_pca[:, 2],
        c=color, cmap='viridis', s=50, alpha=0.8
    )
    ax_3d.set_xlabel('PC1')
    ax_3d.set_ylabel('PC2')
    ax_3d.set_zlabel('PC3')
    ax_3d.set_title('PCA – 3D-Darstellung der ersten drei Hauptkomponenten')

    # Untere Plots: alle 2D-Projektionen der drei PCs
    for i, (x_axis, y_axis) in enumerate([(0, 1), (0, 2), (1, 2)]):
        ax = fig.add_subplot(2, 3, i + 4)
        ax.scatter(
            X_pca[:, x_axis], X_pca[:, y_axis],
            c=color, cmap='viridis', s=40, alpha=0.8
        )
        ax.set_xlabel(f'PC{x_axis + 1}')
        ax.set_ylabel(f'PC{y_axis + 1}')
        ax.grid(True)

    # Abstand optimieren
    plt.tight_layout()
    plt.show()

# Aufruf der Funktion (kompatibel mit DataFrame oder NumPy-Array)
plot_combined_3d(X_pca.values if isinstance(X_pca, pd.DataFrame) else X_pca, color)


**<span style="font-size: 18px; color: orange;">Optional: Interaktive Visualisierung mit Plotly</span>**

Die Bibliothek **Plotly** ermöglicht interaktive Diagramme, in denen man z. B. zoomen, rotieren und Datenpunkte dynamisch untersuchen kann.

Um Plotly zu verwenden, muss es zunächst installiert werden (falls noch nicht geschehen):

```bash
pip install plotly


In [ ]:
import plotly.graph_objects as go

# Interaktiver 3D-Scatterplot mit Plotly
fig = go.Figure(data=[go.Scatter3d(
    x=X_pca['PC1'],
    y=X_pca['PC2'],
    z=X_pca['PC3'],
    mode='markers',
    marker=dict(
        color=color,             # Farbzuordnung über Klassenlabels
        colorscale='viridis',    # Farbschema
        size=5,                  # Punktgröße
        opacity=0.8              # leichte Transparenz für Überlappung
    )
)])

# Layout-Anpassung: Achsentitel, gleiches Seitenverhältnis
fig.update_layout(
    scene=dict(
        xaxis_title='PC1',
        yaxis_title='PC2',
        zaxis_title='PC3',
        aspectmode='cube'  # gleichmäßiges Seitenverhältnis
    ),
    title='PCA – Interaktiver 3D-Scatterplot mit Plotly',
    margin=dict(l=0, r=0, b=0, t=40)
)

# Plot anzeigen
fig.show()


## 2.3 Kombination PCA und Clustering

## <font color='red'>Aufgabe:</font>

Die Kombination aus **PCA** und **Clustering** ermöglicht es, Muster in hochdimensionalen Daten zu erkennen, die in der Originaldarstellung nur schwer sichtbar sind.

Zunächst wird die Dimensionalität der Daten mit **PCA** reduziert. Anschließend wird ein **Clustering-Verfahren** (z. B. KMeans) auf den transformierten Daten (`X_pca`) angewendet.

📌 **Ziel:**  
Führe das Clustering nicht auf den Originaldaten `X`, sondern auf den **dimensionsreduzierten PCA-Daten `X_pca`** durch.  
Die resultierenden Cluster-Labels (`kmeans.predict(...)`) sollen zur **farblichen Darstellung im Scatterplot** verwendet werden.

---

### ✅ Checkpoints:

- [ ] **Daten laden** (`X`, `target`)  
- [ ] **PCA anwenden** (`X_pca` mit 3 Hauptkomponenten)  
- [ ] **KMeans auf `X_pca` anwenden** (Cluster-Labels berechnen)  
- [ ] **Scatterplot erstellen** (z. B. mit Plotly oder Matplotlib)  
- [ ] **Clustering-Ergebnis interpretieren** (visuell oder mit Metriken wie Silhouette Score)
